In [2]:
!pip install contractions
!pip install langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=37c8b3802c0bab5bc294da2d57638da787e23bcfa04c1deab96aebdcdee7a8ec
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [3]:
import os
import pandas as pd
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Downloading the required wordnet data
nltk.download('punkt')
from langdetect import detect

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])
def tokenize_text(text):
    return word_tokenize(text)

In [5]:
def preprocess(df):
    # Convert 'combined' column to strings
    df['combined'] = df['combined'].astype(str)
    
    # Lowercase
    df['combined'] = df['combined'].str.lower()

    # Expanding Contractions
    df['combined'] = df['combined'].apply(lambda x: contractions.fix(x))

    # Removing the URLs
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x, flags=re.MULTILINE))

    # Removing the HTML tags
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'<.*?>', '', x))

    # Removing the numbers
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'\d+', '', x))

    # Removing the extra whitespaces
    df['combined'] = df['combined'].apply(lambda x: x.strip())

    # Removing the punctuation
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

    # Removing the stopwords
    df['combined'] = df['combined'].apply(remove_stopwords)

    # Tokenization
    df['combined'] = df['combined'].apply(tokenize_text)

    return df

# XgBoost

In [6]:
!pip install xgboost

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
import ast
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report


In [8]:
train = pd.read_csv('/kaggle/input/dataset-cleaned/UTK/clean_train_UTK.csv')

In [9]:
train = preprocess(train)

In [10]:
train.head()

,id,title,text,label,combined
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1,"[house, dem, aide, even, see, comeys, letter, ..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0,"[flynn, hillary, clinton, big, woman, campus, ..."
2,2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1,"[truth, might, get, fired, truth, might, get, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1,"[civilian, killed, single, airstrike, identifi..."
4,4,Iranian woman jailed for fictional unpublished...,Print \r\nAn Iranian woman has been sentenced ...,1,"[iranian, woman, jailed, fictional, unpublishe..."


In [11]:
X_train = train['combined']
y_train = train['label']

In [12]:
X_train.head()

0    [house, dem, aide, even, see, comeys, letter, ...
1    [flynn, hillary, clinton, big, woman, campus, ...
2    [truth, might, get, fired, truth, might, get, ...
3    [civilian, killed, single, airstrike, identifi...
4    [iranian, woman, jailed, fictional, unpublishe...
Name: combined, dtype: object

In [13]:
X_train = X_train.apply(' '.join)

In [14]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

# Define a pipeline combining a text feature extractor with a simple classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', tree_method="hist", device="cuda")),
])

# Define the grid parameters
params = {
    'clf__n_estimators': [100, 150],  # number of trees
    'clf__max_depth': [8, 10, 12],  # maximum depth of the trees
    'clf__learning_rate': [0.01, 0.1],  # learning rate
}

# Create a StratifiedKFold object
stratified_kfold = StratifiedKFold(n_splits=5)

# Create the grid search object
grid_search = GridSearchCV(pipeline, params, cv=stratified_kfold, n_jobs=-1, refit=True, verbose=2)

# Fit the grid search object to the data to compute the optimal model
grid_search.fit(X_train, y_train)

# Print the best score and the best parameters
print("Best Score: ", grid_search.best_score_)
print("Best Parameters: ", grid_search.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [04:26:45] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [04:26:49] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for boo

Best Score:  0.6684220865863958
Best Parameters:  {'clf__learning_rate': 0.01, 'clf__max_depth': 10, 'clf__n_estimators': 100}


In [24]:
clf = grid_search.best_estimator_

In [26]:
# Training performance
y_train_pred = clf.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.63      0.86      0.73     10211
           1       0.70      0.40      0.51      8367

    accuracy                           0.65     18578
   macro avg       0.67      0.63      0.62     18578
weighted avg       0.66      0.65      0.63     18578



# Submit test prediction

In [27]:
test = pd.read_csv('/kaggle/input/dataset-cleaned/test.csv')

In [28]:
# Combine `text` and `title` columns into one column
test['combined'] = test['title'] + ' ' + test['text']

In [29]:
test.fillna('', inplace=True)
test = preprocess(test)
X_test = test['combined']
X_test = X_test.apply(' '.join)

In [31]:
id_test = test['id']
y_test_pred = grid_search.predict(X_test)

[CV] END clf__learning_rate=0.01, clf__max_depth=8, clf__n_estimators=100; total time=  35.4s
[CV] END clf__learning_rate=0.01, clf__max_depth=8, clf__n_estimators=150; total time=  43.9s
[CV] END clf__learning_rate=0.01, clf__max_depth=10, clf__n_estimators=100; total time=  25.5s
[CV] END clf__learning_rate=0.01, clf__max_depth=10, clf__n_estimators=150; total time=  27.1s
[CV] END clf__learning_rate=0.01, clf__max_depth=12, clf__n_estimators=100; total time=  25.8s
[CV] END clf__learning_rate=0.01, clf__max_depth=12, clf__n_estimators=100; total time=  29.6s
[CV] END clf__learning_rate=0.01, clf__max_depth=12, clf__n_estimators=150; total time=  31.2s
[CV] END clf__learning_rate=0.1, clf__max_depth=8, clf__n_estimators=100; total time=  19.5s
[CV] END clf__learning_rate=0.1, clf__max_depth=8, clf__n_estimators=150; total time=  20.7s
[CV] END clf__learning_rate=0.1, clf__max_depth=10, clf__n_estimators=100; total time=  20.1s
[CV] END clf__learning_rate=0.1, clf__max_depth=10, clf__

In [32]:
# Create a DataFrame
submission_df = pd.DataFrame({'id': id_test, 'label': y_test_pred})
# Save the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)


In [33]:
# Create a DataFrame
submission_df = pd.DataFrame({'id': id_test, 'label': y_test_pred})
# Save the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

In [34]:
from joblib import dump
dump(clf, 'xgboost_model.joblib')

['xgboost_model.joblib']